Perguntas:
- Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?
- Quantos gêneros cada filme tem, em média?
- Quais são os cinco filmes com maior número de gêneros?
- Quantos usuários deram nota para o filme "The Matrix"?
- Quantos usuários, em média, deram nota para cada filme?
- Quais são os cinco filmes para os quais mais usuários deram nota?
- Qual o rating médio do filme "The Matrix"?
- Cada filme tem um rating médio. Qual a média desses ratings, por gênero?
- Para cada gênero de filme, quantos usuários deram cada nota possível?

Dicas:
1. Só é necessário as tabelas movies e ratings para este projeto.
2. Pesquise a função "explode" no pandas. Ela pode ser útil.

## Importação das Bibliotecas

In [1]:
import pandas as pd
import numpy as np

## Extração dos Dados

In [2]:
def read_data_csv(filename=str, separator=str, data_types=None):
    df = pd.read_csv(filename, sep=separator, decimal='.', dtype=data_types)
    return df

In [112]:
path_dir = 'dados/movie_lens'
df_movies = read_data_csv(f'{path_dir}/movies.csv', ',')
df_ratings = read_data_csv(f'{path_dir}/ratings.csv', ',')

In [113]:
df_movies["ano"] = df_movies["title"].apply(lambda st: st[st.find("(")+1:st.find(")")])
df_movies["title"] = df_movies["title"].str.replace(r" \(.*\)","", regex=True)
df_movies['genres_count'] = df_movies['genres'].str.split('|').str.len()

In [114]:
id_title=df_movies[['movieId','title', 'genres']]
movies_ratings = pd.merge(df_ratings, id_title, how = 'left', on = 'movieId')

In [160]:
movies_genres = df_movies.copy()
movies_genres['genres'] = movies_genres['genres'].str.split('|')
movies_genres = movies_genres.explode('genres')
movies_genres

,movieId,title,genres,ano,genres_count
0,1,Toy Story,Adventure,1995,5
0,1,Toy Story,Animation,1995,5
0,1,Toy Story,Children,1995,5
0,1,Toy Story,Comedy,1995,5
0,1,Toy Story,Fantasy,1995,5
...,...,...,...,...,...
9738,193583,No Game No Life: Zero,Fantasy,2017,3
9739,193585,Flint,Drama,2017,1
9740,193587,Bungo Stray Dogs: Dead Apple,Action,2018,2
9740,193587,Bungo Stray Dogs: Dead Apple,Animation,2018,2


## Exploração dos Dados

### Quantos gêneros o filme "The Matrix" tem? Quais são os gêneros?

In [133]:
df_aux = df_movies.copy()
df_aux["title"] = df_movies['title'].str.replace('[^A-Za-z0-9]+','', regex=True)
df_aux["title"] = df_aux["title"].str.lower()

In [144]:
matrix_id = df_aux[(df_aux["title"]== ("matrixthe"))]
matrix_id = matrix_id.iloc[0]['movieId']
df_aux[(df_aux['movieId'] == matrix_id)]['genres_count'].to_frame("Qtd_genres_matrix")

,Qtd_genres_matrix
1939,3


### Quantos gêneros cada filme tem, em média?

In [145]:
genres_mean = df_movies["genres_count"].mean()
genres_mean

2.2668856497639087

### Quais são os cinco filmes com maior número de gêneros?

In [147]:
genres_highest = df_movies.nlargest(5, "genres_count")
genres_highest = genres_highest["title"].to_frame("Movies")
genres_highest

,Movies
7441,Rubber
5556,Patlabor: The Movie
1390,Mulan
2250,Who Framed Roger Rabbit?
3460,Osmosis Jones


### Quantos usuários deram nota para o filme "The Matrix"?

In [148]:
ratings_matrix_count = movies_ratings[(movies_ratings["movieId"] == matrix_id)]
ratings_matrix_count.count()["rating"]

278

### Quantos usuários, em média, deram nota para cada filme?

In [149]:
users_aver_movie = movies_ratings.groupby("title")["userId"].count().to_frame("users_average")
users_aver_movie['users_average'].mean()

10.730658720868362

### Quais são os cinco filmes para os quais mais usuários deram nota?

In [150]:
movies_ratings.groupby("title")["title"].count().to_frame("total_reviews").nlargest(5, "total_reviews")

,total_reviews
title,
Forrest Gump,329
"Shawshank Redemption, The",317
Pulp Fiction,307
"Silence of the Lambs, The",279
"Matrix, The",278


### Qual o rating médio do filme "The Matrix"?

In [151]:
ratings_matrix_mean = movies_ratings[(movies_ratings["movieId"] == matrix_id)]

In [152]:
ratings_matrix_mean

,userId,movieId,rating,timestamp,title,genres
166,1,2571,5.0,964981888,"Matrix, The",Action|Sci-Fi|Thriller
442,4,2571,1.0,945173385,"Matrix, The",Action|Sci-Fi|Thriller
1130,10,2571,0.5,1455356378,"Matrix, The",Action|Sci-Fi|Thriller
1367,13,2571,5.0,987457007,"Matrix, The",Action|Sci-Fi|Thriller
1475,15,2571,4.0,1510571744,"Matrix, The",Action|Sci-Fi|Thriller
...,...,...,...,...,...,...
96689,603,2571,5.0,963177606,"Matrix, The",Action|Sci-Fi|Thriller
97791,606,2571,5.0,1171325929,"Matrix, The",Action|Sci-Fi|Thriller
98603,607,2571,5.0,963080047,"Matrix, The",Action|Sci-Fi|Thriller
99065,608,2571,5.0,1117162423,"Matrix, The",Action|Sci-Fi|Thriller


In [153]:
ratings_matrix_mean["rating"].mean()

4.192446043165468

### Cada filme tem um rating médio. Qual a média desses ratings, por gênero?

In [154]:
movies_genres = df_movies.copy()
movies_genres.drop(['genres_count'], inplace=True, axis=1)
movies_genres['genres'] = movies_genres['genres'].str.split('|')
movies_genres = movies_genres.explode('genres')
movies_genres

,movieId,title,genres,ano
0,1,Toy Story,Adventure,1995
0,1,Toy Story,Animation,1995
0,1,Toy Story,Children,1995
0,1,Toy Story,Comedy,1995
0,1,Toy Story,Fantasy,1995
...,...,...,...,...
9738,193583,No Game No Life: Zero,Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action,2018
9740,193587,Bungo Stray Dogs: Dead Apple,Animation,2018


In [155]:
mean_by_genres = movies_ratings.groupby("movieId")["rating"].mean().to_frame("rating_mean")
mean_by_genres = pd.merge(movies_genres, mean_by_genres, how = 'left', on = 'movieId')
mean_by_genres.groupby("genres")["rating_mean"].mean().to_frame("mean_genres")

,mean_genres
genres,
(no genres listed),3.371148
Action,3.094498
Adventure,3.215230
Animation,3.497119
Children,3.107690
Comedy,3.181716
Crime,3.301844
Documentary,3.781682
Drama,3.420905


### Para cada gênero de filme, quantos usuários deram cada nota possível?

In [156]:
movies_ratings

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men,Comedy|Romance
2,1,6,4.0,964982224,Heat,Action|Crime|Thriller
3,1,47,5.0,964983815,Seven,Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split,Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two,Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Get Out,Horror
100834,610,168252,5.0,1493846352,Logan,Action|Sci-Fi
